In [ ]:
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import Ollama  # Solution locale
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
import gradio as gr
import torch
import warnings
warnings.filterwarnings('ignore')
import tqdm as notebook_tqdm
from langchain.prompts import PromptTemplate

c:\Users\Axel\anaconda3\envs\mini_rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:

pdf_files = ["cv_fr.pdf", "Valentin_Kocijancic_CV_2025 .pdf", "CV_2025-10-16_Axel_CLEMENT_v2.pdf"]

# chargement pdf
documents = []
for file in pdf_files:
    loader = PyPDFLoader(file)
    docs = loader.load()
    documents.extend(docs)

# chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # taille des chunks
    chunk_overlap=200,    # chevauchement pour garder le contexte
    length_function=len
)
chunks = text_splitter.split_documents(documents)

print(f"Nombre de chunks créés : {len(chunks)}")

Nombre de chunks créés : 10


In [10]:
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={'device': 'cpu'}  # 'cuda' si GPU
)

vector_store = FAISS.from_documents(chunks, embedding_model)

vector_store.save_local("faiss_index")

In [11]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

In [12]:
llm = Ollama(
    model="phi3", #mistral
    temperature=0.1,  # Pour des réponses plus factuelles
    num_predict=256   # Limite la longueur des réponses
)

In [13]:
template = """
Tu es un assistant intelligent spécialisé dans l'analyse de CV pour les recruteurs.
Tu dois répondre **en français** de manière claire, concise et professionnelle.

Tu disposes des informations suivantes issues d'une base de CV :
{context}

Consignes :
- Utilise uniquement les informations présentes dans le contexte pour répondre.
- Si la réponse n'est pas clairement indiquée, dis simplement : "L'information n'est pas disponible dans les CV."
- Si plusieurs candidats semblent correspondre, mentionne leurs **NOM ET PRÉNOM COMPLET** et explique brièvement pourquoi.
- Si un candidat se démarque particulièrement, indique-le clairement en donnant son **NOM ET PRÉNOM COMPLET** et justifie ton choix.

Question du recruteur : {question}

Réponse :
"""

prompt_fr = PromptTemplate(
    input_variables=["context", "question"],
    template=template
)

In [14]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt_fr}
)

In [8]:
question = "Quel candidat cherche un stage en finance ?"
response = qa_chain({"query" : question})
print(response['result'])

Valentin Kocijancic est le candidat qui recherche un stage en finance, comme indiqué dans son CV sous l'objectif. Il souhaite appliquer ses compétences techniques et analytiques au sein d'une équipe dynamique et stimulante pour une période de quatre mois à partir du 13 avril 2026.
